## 📘 Simple RNN от посимвольной до пословной токенизации

### Подключим необходимые пакеты

In [ ]:
"""
Реализация моделей RNN с посимвольной и пословной токенизацией для генерации текста.

Данный модуль обеспечивает создание, обучение и применение рекуррентных нейронных сетей
для генерации текста на русском языке. Реализованы два подхода: символьная и словесная токенизация.
Основной целью является демонстрация базовых принципов работы рекуррентных моделей
и сравнение эффективности различных методов токенизации.
"""

# Библиотеки для работы с данными и базами данных
import sqlite3
from collections import Counter
from typing import List, Dict, Any, Optional, Union, Tuple

import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Библиотеки для обработки текста
import nltk
from nltk.tokenize import sent_tokenize

# Библиотеки глубокого обучения
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Библиотеки для визуализации и анализа
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Утилиты
from tqdm import tqdm

# Настройка визуализации
seaborn.set(palette='summer')

# Определение устройства для вычислений
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Load Data

В качестве датасета испльзуюся старницы Wiki на русском языке.
wikibooks-dataset

In [3]:
conn = sqlite3.connect('/kaggle/input/wikibooks-dataset/wikibooks.sqlite')

df = pd.read_sql_query("SELECT * FROM ru LIMIT 3300", conn)

### Preprocessing data

Разобьём страницы на тексты длиной 256 символов

In [ ]:
# Извлечение предложений из текстов
sentences = []

for sentence in tqdm(df['body_text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='russian') if len(x) < 256]
        )
    
print("Количество предложений", len(sentences))

100%|██████████| 3300/3300 [00:10<00:00, 319.92it/s]

Количество предложений 120873


### 📚 Токенизация на уровне символов: построение словаря

<details> 
    <summary><em><strong>построение словаря 👈</strong></em></summary>

Теперь мы приступим к важному этапу подготовки данных — **токенизации на уровне символов**. Этот шаг лежит в основе создания языковой модели, поскольку позволяет преобразовать текст в числовую форму, пригодную для обработки нейронной сетью.

#### 🔍 Что происходит в коде?

Давайте разберёмся по шагам, что делает наш блок кода:

1. **Определение `stop_chars`**  
   Мы задаём список знаков препинания и специальных символов, которые хотим исключить из словаря. Это помогает уменьшить его размер и сосредоточиться на более значимых символах.

2. **Подсчёт частоты символов**  
   Используем `Counter` из библиотеки `collections`, чтобы определить, как часто встречается каждый символ в корпусе текста.

3. **Фильтрация и перебор**  
   Проходим по всем предложениям и символам, игнорируя те, что находятся в списке `stop_chars`.

4. **Инициализация словаря (`vocab`)**  
   Создаём начальный словарь, включающий четыре **специальных токена**, о которых поговорим чуть ниже.

5. **Порог отбора (`counter_threshold = 500`)**  
   Устанавливаем порог, согласно которому в словарь войдут только те символы, которые встречаются чаще 500 раз.

6. **Формирование финального словаря**  
   Добавляем в `vocab` только те символы, которые соответствуют установленному порогу.

#### ⚙️ Специальные токены и их роль

Словарь начинается с четырёх **специальных токенов**, которые играют ключевую роль в обучении и генерации текста:

| Токен    | Название                    | Описание |
|----------|-----------------------------|----------|
| `<unk>`  | *Unknown*                   | Используется для неизвестных или редких символов, которых нет в словаре. |
| `<bos>`  | *Beginning of Sequence*     | Обозначает начало последовательности (например, предложения). Помогает модели понять контекст начала текста. |
| `<eos>`  | *End of Sequence*           | Сигнализирует о завершении последовательности. Важно для обучения модели корректно заканчивать генерацию. |
| `<pad>`  | *Padding*                   | Токен заполнения, используется для выравнивания длины последовательностей в пакете. |

Эти токены дают модели возможность работать с различными длинами предложений и адекватно интерпретировать начало и конец текста.

#### 📉 Почему мы используем пороговое значение?

Установка порога (`counter_threshold = 500`) — это стратегическое решение, которое:

- ✅ **Уменьшает размер словаря**, делая модель более компактной и эффективной
- ✅ **Фокусирует обучение на наиболее часто встречающихся символах**
- ✅ **Снижает риск переобучения** на редкие или случайные комбинации
- ✅ **Ускоряет обучение** и уменьшает требования к памяти

Благодаря этому подходу мы сохраняем важную информацию, игнорируя шум и малозначимые детали.

#### 🧱 Итог

Этот этап — основа всей последующей работы модели. Посимвольная токенизация даёт модели возможность учиться на тонких закономерностях текста, таких как пунктуация, регистр, морфология и даже стиль. С правильно построенным словарём и чёткой системой токенов мы готовы двигаться дальше — к созданию самой языковой модели.

</details> 

In [ ]:
# Обработка и токенизация на уровне символов
stop_chars = [
    "\t", ",", ".", "!", "@", "'", '"', ";", "\n", "(", ")",
    "[", "]", "{", "}", "?", ":", "-", "_", "+", "=", "^", "*", 
    "&", "`", "~"
]

chars = Counter()

for sentence in tqdm(sentences):
    for char in sentence:
        if char in stop_chars:
            continue
        chars[char] += 1
        
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
counter_threshold = 500

for char, cnt in chars.items():
    if cnt > counter_threshold:
        vocab.add(char)
        
print("Размер словаря:", len(vocab))

100%|██████████| 120873/120873 [00:08<00:00, 13696.57it/s]

Размер словаря: 91


In [6]:
char2ind = {char: i for i, char in enumerate(vocab)}
ind2char = {i: char for char, i in char2ind.items()}

### 🧪 Подготовка данных для обучения RNN: от текста к тензорам

<details> 
    <summary><em><strong>от текста к тензорам 👈</strong></em></summary>

Теперь, когда у нас есть **словарь символов**, мы можем перейти к следующему этапу — подготовке данных для обучения рекуррентной нейронной сети (RNN). Этот этап преобразует "сырой" текст в последовательности чисел, а затем — в тензоры, с которыми работает модель.

#### 📦 1. Создание специализированного датасета (`CharDataset`)

```python
class CharDataset:
    # ... код класса ...
```

Для удобства работы с данными мы создаём собственный класс `CharDataset`, который:

- ✅ Хранит список предложений и предоставляет к ним доступ по индексу
- ✅ Токенизирует текст "на лету", используя наш словарь
- ✅ Добавляет специальные токены `<bos>` в начало и `<eos>` в конец каждого предложения
- ✅ Заменяет неизвестные символы на `<unk>`, чтобы избежать ошибок

Когда вы вызываете `dataset[i]`, метод `__getitem__` возвращает **последовательность числовых индексов**, соответствующих символам предложения. Это первый шаг к тому, чтобы данные стали "понятны" нейросети.

#### 🔄 2. Упаковка данных в батчи с выравниванием (`collate_fn_with_padding`)

```python
def collate_fn_with_padding(input_batch, pad_id=char2ind['<pad>']):
    # ... код функции ...
```

Поскольку предложения имеют разную длину, нам нужно их **выравнивать** внутри батча. Для этого используется функция `collate_fn_with_padding`, которая:

- 🔁 Определяет максимальную длину последовательности в текущем батче
- ⬛ Добавляет паддинг-токены `<pad>` ко всем коротким последовательностям
- 🧮 Преобразует списки в тензоры PyTorch
- 🎯 Формирует пары `(input_ids, target_ids)` для обучения языковой модели

##### Пример вход-выход:
Если исходное предложение:  
`<bos>привет<eos>`

Тогда:
- `input_ids`: `[<bos>, п, р, и, в, е]`
- `target_ids`: `[п, р, и, в, е, т]`

Модель учится предсказывать **следующий символ** на основе всех предыдущих. Именно так реализуется задача языкового моделирования на уровне символов.

#### 🚀 3. Создание загрузчиков данных

```python
# Разделение данных на обучающую и тестовую выборки
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)

# Создание датасетов и загрузчиков данных
train_dataset = CharDataset(train_sentences)
eval_dataset = CharDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)
```

На этом этапе мы:

1. 📊 Делим данные на обучающую (80%) и валидационную (20%) выборки
2. 📚 Создаем экземпляры `CharDataset` для каждой выборки
3. 🚣‍♂️ Инициализируем `DataLoader`'ы, которые:
   - 📦 Формируют батчи размером 256 предложений
   - 🔀 Перемешивают данные между эпохами
   - 📐 Применяют функцию `collate_fn_with_padding` для обработки батча
   - 📈 Автоматически преобразуют данные в тензоры PyTorch

Эти загрузчики обеспечивают **эффективную подачу данных** в модель во время обучения и оценки.

#### 🧠 Важно запомнить

Весь процесс подготовки данных можно представить как конвейер:

**Текст → Символы → Токены → Индексы → Тензоры**

Эта цепочка позволяет модели эффективно учиться на текстовых данных любого объёма и структуры. Особенно важно то, что вся логика токенизации и выравнивания скрыта внутри датасета и функции `collate_fn`, что делает обучение чистым, модульным и масштабируемым.

</details> 

Определим класс CharDataset для последующей загрузки его в dataloader

In [ ]:
class CharDataset:
    """
    Description:
    ---------------
        Датасет для работы с посимвольной токенизацией текста.

    Args:
    ---------------
        sentences: Список предложений для обработки

    Examples:
    ---------------
        >>> dataset = CharDataset(sentences)
        >>> sample = dataset[0]
        >>> len(sample)
        # Длина токенизированного предложения
    """
    def __init__(self, sentences: List[str]) -> None:
        self.data = sentences
        self.unk_id = char2ind['<unk>']
        self.bos_id = char2ind['<bos>']
        self.eos_id = char2ind['<eos>']
        self.pad_id = char2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Преобразует предложение в последовательность индексов символов.

        Args:
        ---------------
            idx: Индекс предложения в датасете

        Returns:
        ---------------
            List[int]: Токенизированное предложение как список индексов
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            char2ind.get(char, self.unk_id) for char in self.data[idx]
        ]
        tokenized_sentence += [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает размер датасета.

        Returns:
        ---------------
            int: Количество предложений в датасете
        """
        return len(self.data)

Определим функцию дополнения предложений до max_seq_len

In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], 
    pad_id: int = char2ind['<pad>']
) -> Dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для преобразования батча данных: добавляет паддинг и 
        создает тензоры ввода и целевых значений.

    Args:
    ---------------
        input_batch: Пакет токенизированных предложений
        pad_id: Идентификатор токена для заполнения (паддинга)

    Returns:
    ---------------
        Dict[str, torch.Tensor]: Словарь с тензорами входных и целевых id
    """
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch

Создадим dataloader для train и eval датасетов

In [ ]:
# Разделение данных на обучающую и тестовую выборки
train_sentences, eval_sentences = train_test_split(
    sentences, test_size=0.2
)

# Создание датасетов и загрузчиков данных
train_dataset = CharDataset(train_sentences)
eval_dataset = CharDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=256
)

### 🧠 Обучение рекуррентной нейронной сети: от теории к практике

<details> 
    <summary><em><strong>от теории к практике 👈</strong></em></summary>

Теперь, когда данные подготовлены, мы можем приступить к обучению модели. Этот этап — сердце всего процесса создания языковой модели: модель будет учиться предсказывать следующий символ на основе предыдущих.

#### 🎯 Зачем мы это делаем?

Цель обучения — научить модель **понимать** закономерности в последовательностях символов. Модель должна "выучить" не только грамматику и словарный запас, но и стилистику, частотные паттерны и даже логику построения предложений.

#### 🚦 1. Функция `fit_epoch`: обучение на одной эпохе

```python
def fit_epoch(model, train_loader, criterion, optimizer):
    # ... код ...
```

Эта функция реализует **один полный проход** через все данные обучающей выборки (эпоха). Вот основные шаги:

1. **`model.train()`** — активируем режим обучения  
   - Включаются такие механизмы, как Dropout или BatchNorm
   - Включается вычисление градиентов

2. **Обработка батч за батчем**  
   - Данные разбиваются на порции для эффективного использования памяти

3. **Для каждого батча:**
   - ❌ `optimizer.zero_grad()` — очищаем старые градиенты
   - 🔁 Прямое распространение (`model(input_ids)`) — получаем предсказания
   - 📉 Вычисляем loss — сравниваем предсказания с реальными символами
   - ⬅️ `loss.backward()` — вычисляем градиенты
   - ✅ `optimizer.step()` — обновляем веса модели

4. **Отслеживание метрик**
   - Сохраняем значение loss
   - Считаем перплексию: `torch.exp(loss)`  
     Это стандартная метрика качества языковых моделей

> 💡 **Перплексия (Perplexity)** — это мера "неожиданности" текста для модели:
- Перплексия = 1 → модель идеально предсказывает следующий символ
- Перплексия = 10 → модель в среднем рассматривает 10 возможных следующих символов
- Чем ниже — тем лучше!

#### 🔍 2. Функция `eval_epoch`: оценка на валидационной выборке

```python
def eval_epoch(model, val_loader, criterion):
    # ... код ...
```

Функция `eval_epoch` проверяет, **насколько хорошо модель обобщает знания** на новых данных, которые она не видела при обучении.

Основные особенности:
- **`model.eval()`** — переходим в режим оценки:
  - Отключается Dropout
  - Не вычисляются градиенты
- **Контекст `with torch.no_grad()`** — экономим память и повышаем производительность
- **Вычисляем те же метрики**, что и при обучении — чтобы можно было сравнить результаты

##### Почему важна валидация?
- Позволяет **обнаружить переобучение** (когда модель "зубрит", а не "понимает")
- Помогает выбрать **момент остановки** обучения
- Указывает на **реальное качество модели**

#### 🏋️‍♂️ 3. Функция `train`: полный цикл обучения

```python
def train(...):
    # ... код ...
```

Это "дирижёр" всего процесса. Он координирует работу `fit_epoch` и `eval_epoch`, управляет оптимизатором и контролирует сохранение лучшей модели.

Что происходит внутри:

##### 🔧 Настройка инструментов обучения
- Инициализация **оптимизатора** (`Adam`) — алгоритм обновления весов
- Инициализация **функции потерь** (`CrossEntropyLoss`) — способ сравнения предсказаний и целевых значений
- Указание `ignore_index=char2ind['<pad>']` — игнорируем лишние паддинги при обучении

##### 🛑 Early Stopping
- Отслеживаем **лучшую модель** по минимальной перплексии
- Сохраняем **веса лучшей модели**
- Прерываем обучение, если улучшений нет долгое время

##### 📈 Логирование и визуализация
- Сохраняем историю метрик
- Используем `tqdm` для красивого прогресс-бара
- Выводим понятную таблицу результатов после каждой эпохи

#### 🔄 Как всё работает вместе?

Процесс обучения строится как цикл:

```
ПОКА НЕ ДОСТИГНУТО УСЛОВИЕ ОСТАНОВКИ:
    1. Пройти по всем данным (одна эпоха)
    2. Обновить веса модели
    3. Протестировать на валидации
    4. Если результат лучше — сохранить модель
```

По завершении обучения загружается **лучшая версия модели**, найденная за весь процесс.

#### 🧮 Итак, что мы измеряем?

| Метрика       | Что показывает                              | Интерпретация                        |
|---------------|----------------------------------------------|---------------------------------------|
| Loss          | Средняя величина ошибки                      | Меньше — лучше                         |
| Perplexity    | Экспонента от loss                           | Ближе к 1 — лучше                      |

Модель стремится минимизировать loss и, соответственно, перплексию — это и есть **обучение**.

#### 📦 Подводя итог

Весь процесс обучения можно представить как:

**Данные → Прогноз → Ошибка → Обновление → Лучший прогноз**

Модель учится "предугадывать" следующий символ, опираясь на предыдущие. Так, шаг за шагом, она осваивает язык и становится способной не только воспроизводить, но и **генерировать** текст.

</details> 

### Train loop

In [ ]:
def fit_epoch(
    model: nn.Module, 
    train_loader: DataLoader, 
    criterion: nn.Module, 
    optimizer: torch.optim.Optimizer, 
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> Tuple[float, float]:
    """
    Description:
    ---------------
        Выполняет одну эпоху обучения модели.

    Args:
    ---------------
        model: Модель для обучения
        train_loader: Загрузчик обучающих данных
        criterion: Функция потерь
        optimizer: Оптимизатор
        sheduler: Планировщик скорости обучения (опционально)

    Returns:
    ---------------
        Tuple[float, float]: Перплексия и значение функции потерь
    """
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    losses = []
    perplexity = []
    
    for batch in train_loader:
        optimizer.zero_grad()

        # Распространение прямое и обратное
        logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
        loss = criterion(
            logits, batch['target_ids'].flatten()
        )
        loss.backward()
        optimizer.step()
        
        # Сохранение метрик
        perplexity.append(torch.exp(loss).item())
        losses.append(loss.item())
        
    # Расчет средних значений метрик
    perplexity_avg = sum(perplexity) / len(perplexity)
    losses_avg = sum(losses) / len(losses)    
    
    return perplexity_avg, losses_avg

In [ ]:
def eval_epoch(
    model: nn.Module, 
    val_loader: DataLoader, 
    criterion: nn.Module
) -> Tuple[float, float]:
    """
    Description:
    ---------------
        Оценивает модель на валидационном наборе данных.

    Args:
    ---------------
        model: Модель для оценки
        val_loader: Загрузчик валидационных данных
        criterion: Функция потерь

    Returns:
    ---------------
        Tuple[float, float]: Перплексия и значение функции потерь
    """
    model.eval()
    perplexity = []
    losses = []
    
    with torch.no_grad():
        for batch in val_loader:
            logits = model(batch['input_ids']).flatten(start_dim=0, end_dim=1)
            loss = criterion(
                logits,
                batch['target_ids'].flatten()
            )
            perplexity.append(torch.exp(loss).item())
            losses.append(loss.item())

    # Расчет средних значений метрик
    perplexity_avg = sum(perplexity) / len(perplexity)
    losses_avg = sum(losses) / len(losses)
    
    return perplexity_avg, losses_avg

In [ ]:
def train(
    train_dataloader: DataLoader, 
    eval_dataloader: DataLoader, 
    model: nn.Module, 
    epochs: int, 
    ignore_index: int = char2ind['<pad>'],
    optimizer: Optional[torch.optim.Optimizer] = None, 
    criterion: Optional[nn.Module] = None, 
    sheduler: Optional[torch.optim.lr_scheduler._LRScheduler] = None
) -> Tuple[nn.Module, List[Tuple[float, float, float, float]]]:
    """
    Description:
    ---------------
        Обучает модель на заданном количестве эпох.

    Args:
    ---------------
        train_dataloader: Загрузчик обучающих данных
        eval_dataloader: Загрузчик валидационных данных
        model: Модель для обучения
        epochs: Количество эпох обучения
        ignore_index: Индекс токена, который не учитывается в функции потерь
        optimizer: Оптимизатор (по умолчанию Adam)
        criterion: Функция потерь (по умолчанию CrossEntropyLoss)
        sheduler: Планировщик скорости обучения (опционально)

    Returns:
    ---------------
        Tuple[nn.Module, List[Tuple[float, float, float, float]]]: 
            Обученная модель и история обучения
    """
    # Инициализация оптимизатора и функции потерь, если не указаны
    if optimizer is None:
      optimizer = torch.optim.Adam(model.parameters())

    if criterion is None:
      criterion = nn.CrossEntropyLoss(ignore_index=ignore_index)

    # Сохранение лучших весов модели
    best_model_wts = model.state_dict()
    best_perplexity = 10e10

    # История обучения
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_perplexirty {t_acc:0.4f} val_perplexirty {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        for epoch in range(epochs):
            # Обучение на одной эпохе
            train_perplexirty, train_loss = fit_epoch(
                model, train_dataloader, criterion, optimizer
            )
            
            # Валидация модели
            val_perplexirty, val_loss = eval_epoch(
                model, eval_dataloader, criterion
            )
            
            # Сохранение метрик
            history.append((train_loss, train_perplexirty, val_loss, val_perplexirty))
            
            # Сохранение лучшей модели
            if val_perplexirty < best_perplexity:
                best_perplexity = val_perplexirty
                best_model_wts = model.state_dict()

            # Обновление прогресс-бара и вывод результатов
            pbar_outer.update(1)
            tqdm.write(log_template.format(
                ep=epoch+1, 
                t_loss=train_loss,
                v_loss=val_loss, 
                t_acc=train_perplexirty, 
                v_acc=val_perplexirty
            ))

    print('Best val perplexirty: {:4f}'.format(best_perplexity))
    
    # Загрузка лучших весов
    model.load_state_dict(best_model_wts)

    return model, history

### 🧠 Архитектура языковой модели: Simple RNN на базе GRU

<details> 
    <summary><em><strong>Simple RNN 👈</strong></em></summary>

Теперь давайте посмотрим на **архитектуру нашей языковой модели**. Мы будем использовать рекуррентную нейронную сеть (RNN), а точнее её современную реализацию — **GRU (Gated Recurrent Unit)**. Это позволяет модели эффективно работать с последовательностями и "запоминать" контекст.

#### 🔨 Основной класс модели `CharLM`

```python
class CharLM(nn.Module):
    # ... код класса ...
```

Это наша модель для **посимвольного языкового моделирования**. Она состоит из нескольких ключевых слоёв:

---

#### 1. Слой эмбеддингов (`self.embedding`)

```python
self.embedding = nn.Embedding(vocab_size, hidden_dim)
```

- **Что делает**: преобразует индексы символов в плотные векторы (эмбеддинги)
- **Зачем нужно**: позволяет модели работать с символами как с числами в многомерном пространстве, где семантически близкие символы будут расположены рядом
- **Вход/Выход**:
  - Вход: `[batch_size, seq_len]`
  - Выход: `[batch_size, seq_len, hidden_dim]`

---

#### 2. Рекуррентный слой GRU (`self.rnn`)

```python
self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
```

- **Что делает**: обрабатывает последовательности, сохраняя информацию о предыдущих символах
- **Как работает**:
  - Использует **update gate** и **reset gate**, чтобы контролировать поток информации
  - Может запоминать зависимости на длинных последовательностях
- **Преимущества над обычным RNN**:
  - Лучше справляется с проблемой исчезающих градиентов
  - Более устойчив к долгосрочным зависимостям
- **Параметр `batch_first=True`**: говорит PyTorch, что первый размер — это размер батча

---

#### 3. Дополнительные слои обработки

```python
self.linear = nn.Linear(hidden_dim, hidden_dim)
self.non_lin = nn.Tanh()
self.dropout = nn.Dropout(p=0.1)
```

- **Линейный слой (`linear`)**: тонкая настройка скрытых представлений
- **Нелинейность (`Tanh`)**: добавляет нелинейность в модель, увеличивая её выразительность
- **Dropout (`p=0.1`)**: уменьшает переобучение, случайно "отключая" часть нейронов во время обучения

---

#### 4. Проекционный слой (`self.projection`)

```python
self.projection = nn.Linear(hidden_dim, vocab_size)
```

- **Что делает**: преобразует скрытое состояние сети в логиты — значения, из которых затем строится распределение вероятностей по всем символам словаря
- **Размерность выхода**: `[batch_size, seq_len, vocab_size]`
- **Итог**: для каждой позиции в последовательности мы получаем вероятности всех возможных следующих символов

---

#### 🔄 Прямой проход (`forward`)

Метод `forward` определяет, как данные проходят через модель:

1. **Эмбеддинги**: индексы превращаются в векторы
2. **GRU**: обработка последовательности с учётом контекста
3. **Дополнительная обработка**: линейное преобразование + Tanh + Dropout
4. **Проекция**: получаем логиты — ненормированные вероятности следующего символа

---

#### 💡 Почему именно такая архитектура?

Эта модель представляет собой **классический пример рекуррентной сети для языкового моделирования**. Вот почему она работает хорошо:

| Компонент      | Зачем он нужен |
|----------------|----------------|
| GRU            | хорошо работает с последовательностями, запоминает контекст |
| Эмбеддинги     | представляют символы в числовом виде |
| Dropout        | защищает от переобучения |
| Дополнительные слои | увеличивают выразительность модели |
| Размерность 256 | баланс между скоростью и качеством |

---

#### 📦 Что получаем в итоге?

Модель, которая:
- ✅ Может обучаться на больших объёмах текста
- ✅ Улавливает закономерности на уровне символов
- ✅ Генерирует текст, напоминающий обучающие данные
- ✅ Имеет около **296 тысяч параметров** — достаточно компактна, но при этом эффективна

---

#### 🧩 На заметку

Хотя современные модели (например, Transformer) значительно превосходят RNN по качеству, понимание работы рекуррентных сетей остаётся важным этапом в освоении NLP. GRU — отличная отправная точка для изучения последовательного моделирования.

</details>

### Main model

In [ ]:
class CharLM(nn.Module):
    """
    Description:
    ---------------
        Модель языковой модели на базе GRU для посимвольной генерации.

    Args:
    ---------------
        hidden_dim: Размерность скрытого состояния
        vocab_size: Размер словаря

    Examples:
    ---------------
        >>> model = CharLM(hidden_dim=256, vocab_size=1000)
        >>> input_tensor = torch.LongTensor([[1, 2, 3, 4]])
        >>> output = model(input_tensor)
        >>> output.shape
        torch.Size([1, 4, 1000])
    """
    def __init__(self, hidden_dim: int, vocab_size: int) -> None:
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.non_lin = nn.Tanh()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        """
        Description:
        ---------------
            Прямой проход модели.

        Args:
        ---------------
            input_batch: Тензор с индексами входных символов

        Returns:
        ---------------
            torch.Tensor: Предсказанные логиты для каждого символа
        """
        # Преобразование индексов в эмбеддинги
        embeddings = self.embedding(input_batch)  # [batch_size, seq_len, hidden_dim]
        
        # Пропуск через рекуррентный слой
        output, _ = self.rnn(embeddings)  # [batch_size, seq_len, hidden_dim]
        
        # Дополнительные слои
        output = self.dropout(
            self.linear(self.non_lin(output))
        )  # [batch_size, seq_len, hidden_dim]
        
        # Проекция на размер словаря
        projection = self.projection(
            self.non_lin(output)
        )  # [batch_size, seq_len, vocab_size]

        return projection

In [ ]:
# Инициализация и обучение модели на посимвольных данных
model = CharLM(hidden_dim=256, vocab_size=len(vocab)).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

CharLM(
  (embedding): Embedding(91, 256)
  (rnn): GRU(256, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=91, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
)
Number of model parameters: 507,227


In [15]:
model, history = train(train_dataloader, eval_dataloader, model, 10)

epoch:  10%|█         | 1/10 [00:35<05:15, 35.02s/it]


Epoch 001 train_loss: 2.1936     val_loss 1.7596 train_perplexirty 10.3569 val_perplexirty 5.8121


epoch:  20%|██        | 2/10 [01:09<04:36, 34.51s/it]


Epoch 002 train_loss: 1.6882     val_loss 1.6169 train_perplexirty 5.4163 val_perplexirty 5.0392


epoch:  30%|███       | 3/10 [01:43<04:00, 34.33s/it]

...


Epoch 008 train_loss: 1.4678     val_loss 1.4623 train_perplexirty 4.3410 val_perplexirty 4.3171


epoch:  90%|█████████ | 9/10 [05:08<00:34, 34.15s/it]


Epoch 009 train_loss: 1.4568     val_loss 1.4540 train_perplexirty 4.2933 val_perplexirty 4.2817


epoch: 100%|██████████| 10/10 [05:42<00:00, 34.24s/it]


Epoch 010 train_loss: 1.4475     val_loss 1.4468 train_perplexirty 4.2539 val_perplexirty 4.2509
Best val perplexirty: 4.250897


Функция для генерации последовательности

In [ ]:
def generate_sequence(
    model: nn.Module, 
    dict_2ind: Dict[str, int], 
    ind2dict: Dict[int, str], 
    starting_seq: str, 
    max_seq_len: int = 256
) -> str:
    """
    Description:
    ---------------
        Генерирует текстовую последовательность, начиная с заданной.

    Args:
    ---------------
        model: Обученная языковая модель
        dict_2ind: Словарь для преобразования символов в индексы
        ind2dict: Словарь для преобразования индексов в символы
        starting_seq: Начальная последовательность
        max_seq_len: Максимальная длина генерируемой последовательности

    Returns:
    ---------------
        str: Сгенерированная последовательность

    Examples:
    ---------------
        >>> result = generate_sequence(model, char2ind, ind2char, "привет")
        >>> print(result)
        'привет мир...'
    """
    device = 'cpu'
    model = model.to(device)
    
    # Преобразование начальной последовательности в индексы
    input_ids = [dict_2ind['<bos>']] + [
        dict_2ind.get(char, dict_2ind['<unk>']) for char in starting_seq
    ]
    input_ids = torch.LongTensor(input_ids).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            # Получение распределения вероятностей для следующего символа
            next_char_distribution = model(input_ids.unsqueeze(0))
            next_char_logits = next_char_distribution[0, -1, :]
            next_char = next_char_logits.argmax()
            
            # Добавление предсказанного символа к входной последовательности
            input_ids = torch.cat([input_ids, next_char.unsqueeze(0)])

            # Завершение генерации при появлении токена конца последовательности
            if next_char.item() == dict_2ind['<eos>']:
                break

    # Преобразование индексов обратно в символы
    words = ' '.join([ind2dict[idx.item()] for idx in input_ids])

    return words

In [ ]:
# Генерация примера текста
generate_sequence(model, char2ind, ind2char, starting_seq='источник ')

'<bos>источник может быть производственные программирования и программирования программирования в программировании в программировании в программировании в программировании в программировании в программировании в программировании в программировании в программировании в пр'

### Токенизация по словам

In [ ]:
# Очистка памяти
import gc
torch.cuda.empty_cache()
gc.collect()

1086

In [ ]:
# Повторное извлечение предложений
sentences = []

for sentence in tqdm(df['body_text']):
    sentences.extend(
        [x.lower() for x in sent_tokenize(sentence, language='russian') if len(x) < 256]
    )
    
print("Количество предложений", len(sentences))

# Обработка и токенизация на уровне слов
words = Counter()

for sentence in tqdm(sentences):
    for word in nltk.word_tokenize(sentence):
            words[word] += 1
            
vocab = set(['<unk>', '<bos>', '<eos>', '<pad>'])
vocab_size = 40000

for elem in words.most_common(vocab_size):
    vocab.add(elem[0])
    
print("Всего слов в словаре:", len(vocab))

100%|██████████| 3300/3300 [00:10<00:00, 319.10it/s]


Количество предложений 120873


100%|██████████| 120873/120873 [00:29<00:00, 4159.08it/s]


Всего слов в словаре: 40004


In [22]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [ ]:
class WordDataset:
    """
    Description:
    ---------------
        Датасет для работы с пословной токенизацией текста.

    Args:
    ---------------
        sentences: Список предложений для обработки

    Examples:
    ---------------
        >>> dataset = WordDataset(sentences)
        >>> sample = dataset[0]
        >>> len(sample)
        # Длина токенизированного предложения
    """
    def __init__(self, sentences: List[str]) -> None:
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        """
        Description:
        ---------------
            Преобразует предложение в последовательность индексов слов.

        Args:
        ---------------
            idx: Индекс предложения в датасете

        Returns:
        ---------------
            List[int]: Токенизированное предложение как список индексов
        """
        tokenized_sentence = [self.bos_id]
        tokenized_sentence += [
            word2ind.get(word, self.unk_id) 
            for word in nltk.word_tokenize(self.data[idx])
        ]
        tokenized_sentence += [self.eos_id]
        
        return tokenized_sentence

    def __len__(self) -> int:
        """
        Description:
        ---------------
            Возвращает размер датасета.

        Returns:
        ---------------
            int: Количество предложений в датасете
        """
        return len(self.data)

In [ ]:
def collate_fn_with_padding(
    input_batch: List[List[int]], 
    pad_id: int = word2ind['<pad>']
) -> Dict[str, torch.Tensor]:
    """
    Description:
    ---------------
        Функция для преобразования батча данных: добавляет паддинг и 
        создает тензоры ввода и целевых значений.

    Args:
    ---------------
        input_batch: Пакет токенизированных предложений
        pad_id: Идентификатор токена для заполнения (паддинга)

    Returns:
    ---------------
        Dict[str, torch.Tensor]: Словарь с тензорами входных и целевых id
    """
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:, :-1],
        'target_ids': sequences[:, 1:]
    }

    return new_batch

In [ ]:
# Разделение данных на обучающую и тестовую выборки
train_sentences, eval_sentences = train_test_split(
    sentences, test_size=0.2
)

# Создание датасетов и загрузчиков данных для словесной модели
train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)

train_dataloader = DataLoader(
    train_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=64
)

eval_dataloader = DataLoader(
    eval_dataset, 
    collate_fn=collate_fn_with_padding, 
    batch_size=64
)

In [ ]:
# Инициализация и обучение модели на словесных данных
model = CharLM(hidden_dim=256, vocab_size=len(vocab)).to(device)

num_params = sum(p.numel() for p in model.parameters())
print(model)
print(f"Number of model parameters: {num_params:,}")

CharLM(
  (embedding): Embedding(40004, 256)
  (rnn): GRU(256, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=256, bias=True)
  (projection): Linear(in_features=256, out_features=40004, bias=True)
  (non_lin): Tanh()
  (dropout): Dropout(p=0.1, inplace=False)
)
Number of model parameters: 20,982,596


In [ ]:
best_model, losses = train(
    train_dataloader, 
    eval_dataloader, 
    model, 
    10, 
    ignore_index=word2ind["<pad>"]
)

epoch:  10%|█         | 1/10 [01:45<15:45, 105.00s/it]


Epoch 001 train_loss: 4.8116     val_loss 5.2675 train_perplexirty 125.5927 val_perplexirty 198.0715


epoch:  20%|██        | 2/10 [03:30<14:00, 105.06s/it]


Epoch 002 train_loss: 4.4128     val_loss 5.2255 train_perplexirty 83.7987 val_perplexirty 190.3353


epoch:  30%|███       | 3/10 [05:15<12:16, 105.16s/it]

...


Epoch 008 train_loss: 3.2282     val_loss 5.5834 train_perplexirty 25.5596 val_perplexirty 275.9186


epoch:  90%|█████████ | 9/10 [15:45<01:44, 104.99s/it]


Epoch 009 train_loss: 3.1171     val_loss 5.6592 train_perplexirty 22.8660 val_perplexirty 298.2022


epoch: 100%|██████████| 10/10 [17:30<00:00, 105.02s/it]


Epoch 010 train_loss: 3.0210     val_loss 5.7391 train_perplexirty 20.7655 val_perplexirty 323.7256
Best val perplexirty: 190.335309


In [ ]:
# Генерация текста с использованием словесной модели
generate_sequence(
    model, 
    word2ind, 
    ind2word, 
    starting_seq=nltk.word_tokenize('кот')
)

'<bos> кот высотой 2 марта 2008 года выборы президента российской федерации выборы уважаемый избиратель ! <eos>'

### Выводы

**Simple RNN**

GRU продемонстрировала базовую способность к генерации текста, GRU не имеет long памяти, как у LSTM (long short-term memory), поэтому на этапе создании модели было ясно, что качество генерации на больших последовательностях будет храмать очень сильно, а на маленьких последовательностях показывать приемлемые результаты. При посимвольной токенизации модель показала, как было сказано ранее, приемлимые результаты при генерации небольшого текста (2-4 слова). По-словная токенизация улучшила семантическую целостность текста, но остались такие же проблемы как и при посимвольной токенизации.

**Плюсы:**
- Простота реализации и быстрая скорость обучения на коротких текстах.
- Меньшая вычислительная сложность по сравнению с более сложными архитектурами.

**Минусы:**
- Плохая работа с длинными зависимостями.
- Посимвольная токенизация может приводить к потере смысла текста.